In [1]:
from ASmartis import * 
from ipywidgets import HBox,VBox,Layout,Label
import numpy as np
from Colors import Color

class Widerstandsnetzwerk(ASmartis):
    
    def __init__(self,parameterList):
        ildeResistance = 99999999
        shortCircuitResistance = 1/ildeResistance
        
        self.possibleComponents = ["ampere_meter","volt_meter","nothing","resistor","lamp","lamp_broken","switch_open","switch_close","shortCircuit","idle"]
        self.dictComponentResistances={"nothing":ildeResistance,"resistor":100,"lamp":0.1,"lamp_broken":ildeResistance,"switch_open":ildeResistance,"switch_close":shortCircuitResistance,"shortCircuit":shortCircuitResistance,"idle":ildeResistance,"volt_meter":ildeResistance/100,"ampere_meter":shortCircuitResistance*100}
        self.componentNetwork_1 = np.array([["switch_close", "nothing","nothing","nothing"], 
                                         ["nothing", "nothing","nothing","nothing"],
                                         ["nothing", "nothing","nothing","nothing"],
                                         ["nothing", "nothing","nothing","nothing"]],dtype=str)
        
        self.componentNetwork_2 = np.array([["nothing","nothing"], 
                                         ["nothing", "nothing"]],dtype=str)
        
        
        self.actualChoice = [0,0]#[row,column]
        self.oldChoice = [0,0]        

        self.lampResistance = self.dictComponentResistances["lamp"]        
        self.resistancesNetwork_1 = np.array([[0, 0,0,0], [0, 0,0,0],[0,0,0,0],[0,0,0,0]],dtype=float)
        self.resistancesNetwork_2 = np.array([[0, 0], [0, 0]],dtype=float)
        
        
        self.resistancesSeries_1 = [0,0,0,0]
        self.resistancesSeries_2 = [0,0]
        self.totalResistance = 0

        self.lampVoltage = 0
        self.voltageSeries_1 = 0
        self.voltageSeries_2 = 0
        self.voltagesNetwork_1 = self.resistancesNetwork_1.copy()
        self.voltagesNetwork_2 = self.resistancesNetwork_2.copy()
        
        
        self.totalCurrent = 0
        self.currentsNetwork_1 = self.resistancesNetwork_1.copy() 
        self.currentsNetwork_2 = self.resistancesNetwork_2.copy()

        super().__init__(parameterList)
        self.updateTime = 0.1
        self.reset()
        
        
    def createButtons(self):        
        self.buttonsObject = Buttons(self)
        self.buttonsObject.newReset("b_reset", 40,40)
        self.buttonsObject.newIntSlider("s_voltage", 0,999,230,"$U$",newReadout=True)
        
        options = ["Amperemeter","Voltmeter","Nichts","Widerstand","Lampe","Lampe (kaputt)","Schalter (offen)","Schalter (geschl.)","Kurzschluss","Leerlauf"]
        dd_options = []
        for i in range(len(options)):        
            dd_options.append([options[i],self.possibleComponents[i]])
        
        self.buttonsObject.newDropdown("dd_component", dd_options, "Bauteil:")  
        self.buttonsObject.newIntSlider("s_resistance", 0,500,0,"$R$",newReadout=False)
        self.buttonsObject.newFloatTextbox("tb_resistance", 0,"")
        self.buttonsObject.disabled("tb_resistance", True)
        self.buttonsObject.linkingButtons("s_resistance","tb_resistance")

        self.currentValueLabel = Label(value = "")
        self.voltageValueLabel = Label(value = "")

        
            
    def createChart(self):        
        self.layers = ["background","0-0","0-1","0-2","0-3","1-0","1-1","1-2","1-3","1-4","1-5",
                       "2-0","2-1","2-2","2-3","2-4","2-5","3-0","3-1","3-2","3-3",
                       "grid","choice","beams","gauges"]        
        self.width  = 1200
        self.height = 600
        self.border = 10
        self.chartObject = Chart(self.layers, self.width+self.border*2, self.height+self.border*2)
        
        self.createComponents()
        
        self.createBackground()
        
        self.buttonsObject.canvasInteraction("canvas", self.chartObject.getChart())
        self.buttonsStatesDict = self.buttonsObject.getStates()
        self.oldButtonsStatesDict = dict(self.buttonsStatesDict)
    
    def createBackground(self):
        currentLayer = "background"
        b = self.border        

        #senkrechte Linien
        self.chartObject.drawLine(currentLayer,b,self.height/6*0.5+b,b,self.height/6*5.5+b)
        self.chartObject.drawLine(currentLayer,800+b,self.height/6*2.5+b,800+b,self.height/6*5.5+b)
        self.chartObject.drawLine(currentLayer,self.width+b,self.height/6*0.5+b,self.width+b,self.height/6*4.5+b)
        
        #waagerechte Linien
        self.chartObject.drawLine(currentLayer,b,self.height/6*0.5+b,self.width+b,self.height/6*0.5+b)
        self.chartObject.drawLine(currentLayer,b,self.height/6*2.5+b,800+b,self.height/6*2.5+b)
        self.chartObject.drawLine(currentLayer,b,self.height/6*3.5+b,self.width+b,self.height/6*3.5+b)
        self.chartObject.drawLine(currentLayer,b,self.height/6*4.5+b,self.width+b,self.height/6*4.5+b)
        self.chartObject.drawLine(currentLayer,b,self.height/6*5.5+b,800+b,self.height/6*5.5+b)  
        
        #Knotenpunkte 
        self.chartObject.changeLayerLook(currentLayer,"fillingColor",Color.LINES.value)  
        self.chartObject.drawCircle(currentLayer,b,self.height/6*(0+2.5)+b,b/2)
        self.chartObject.drawCircle(currentLayer,b,self.height/6*(1+2.5)+b,b/2)
        self.chartObject.drawCircle(currentLayer,b,self.height/6*(2+2.5)+b,b/2)
        self.chartObject.drawCircle(currentLayer,800+b,self.height/6*(1+2.5)+b,b/2)
        self.chartObject.drawCircle(currentLayer,800+b,self.height/6*(2+2.5)+b,b/2)
        self.chartObject.drawCircle(currentLayer,self.width+b,self.height/6*(1+2.5)+b,b/2)
        
        #Spannungsquelle und Lampe
        self.chartObject.drawImage(currentLayer,self.componentDict["lamp"],self.width/6*3+b,b)
        self.chartObject.drawCircle(currentLayer,self.width/6*2+b,self.height/6*0.5+b,self.height/(6*4),False)
        self.chartObject.changeLayerLook(currentLayer,"lineColor",Color.U.value)  
        self.chartObject.changeLayerLook(currentLayer,"fillingColor",Color.U.value)  
        self.chartObject.changeLayerLook(currentLayer,"lineWidth",3)
        self.chartObject.drawArrow(currentLayer,self.width/6*2+b-30,2.5*b,self.width/6*2+b+30,2.5*b,18,10)
        self.chartObject.drawText("background",self.width/6*2+b,2*b,"italic 19px serif", "U","center")

        

        
        
        self.chartObject.update(currentLayer)
        
        
        currentLayer = "grid"
        
        self.chartObject.changeLayerLook(currentLayer,"lineColor",Color.EDGE.value)   
        self.chartObject.changeLayerLook(currentLayer,"lineDash",[3,3])
        
        for i in range(5):
            self.chartObject.drawLine(currentLayer,b,self.height/6*(i+2)+b,800+b,self.height/6*(i+2)+b)
            self.chartObject.drawLine(currentLayer,self.width/6*i+b,self.height/6*2+b,self.width/6*i+b,self.height/6*6+b)
        
        self.chartObject.drawLine(currentLayer,800+b,self.height/6*(1+2)+b,self.width/6*6+b,self.height/6*(1+2)+b)
        self.chartObject.drawLine(currentLayer,800+b,self.height/6*(2+2)+b,self.width/6*6+b,self.height/6*(2+2)+b)
        self.chartObject.drawLine(currentLayer,800+b,self.height/6*(3+2)+b,self.width/6*6+b,self.height/6*(3+2)+b)
        self.chartObject.drawLine(currentLayer,self.width/6*5+b,self.height/6*3+b,self.width/6*5+b,self.height/6*5+b)
        self.chartObject.drawLine(currentLayer,self.width/6*6+b,self.height/6*3+b,self.width/6*6+b,self.height/6*5+b)
        
        self.chartObject.update(currentLayer) 
        
        
        currentLayer = "choice"
        self.chartObject.changeLayerLook(currentLayer,"lineColor",Color.CHOICE.value)   
        self.chartObject.changeLayerLook(currentLayer,"lineDash",[3,3])
        self.chartObject.changeLayerLook(currentLayer,"lineWidth",2)
        self.chartObject.update(currentLayer) 
        
        
        
        
    def visualize(self):   
        chart = self.chartObject.getChart()
        buttonsDict = self.buttonsObject.getButtons()
        
        upperBox = HBox([buttonsDict["s_voltage"],buttonsDict["b_reset"]],layout = Layout(justify_content = "space-around"))
        
        self.buttonsObject.changeLayout(name = "dd_component",width = "30%")
        self.buttonsObject.changeLayout(name = "s_resistance",width = "60%")
        self.buttonsObject.changeLayout(name = "tb_resistance",width = "10%")
        
        currentLabel = Label(value = "$I=$")
        currentUnitLabel = Label(value = "$A$")
        currentBox = HBox([currentLabel,self.currentValueLabel,currentUnitLabel])
                
        voltageLabel = Label(value = "$U=$")
        voltageUnitLabel = Label(value = "$V$")
        voltageBox = HBox([voltageLabel,self.voltageValueLabel,voltageUnitLabel])
        
        buttonsBox = HBox([buttonsDict["dd_component"],buttonsDict["s_resistance"],buttonsDict["tb_resistance"]])
        textBox = HBox([currentBox,voltageBox],layout = Layout(justify_content = "space-around"))
        
        display(VBox([upperBox,chart,buttonsBox,textBox],layout=Layout(width="100%")))
        
        
    def update(self):
        if self.buttonsStatesDict["b_reset"]:
            self.reset()            
        else:  
            if self.buttonsStatesDict["canvas"] != self.oldButtonsStatesDict["canvas"]:
                self.interactionHandling(self.buttonsStatesDict["canvas"][0],self.buttonsStatesDict["canvas"][1])
                if self.actualChoice != self.oldChoice :
                    self.updateChartChoice()
                    self.updateButtons()
                    self.updateLabels()
                    self.oldChoice = self.actualChoice
                    
            elif self.buttonsStatesDict["s_resistance"] != self.oldButtonsStatesDict["s_resistance"]:
                self.setResistanceNetworks()
                self.callculateValues()
                self.updateLabels()
                self.updateGauges()
                self.updateBeams()
                   
            elif self.buttonsStatesDict["s_voltage"] != self.oldButtonsStatesDict["s_voltage"]:
                self.callculateValues()
                self.updateLabels()
                self.updateGauges()
                self.updateBeams()
                
            elif self.buttonsStatesDict["dd_component"] != self.oldButtonsStatesDict["dd_component"]:                 
                self.setComponentNetworks()
                self.updateCircuit()
                self.callculateValues()
                self.updateButtons()
                self.updateLabels()
                self.updateGauges()
                self.updateBeams()

            else:
                pass
                
    def reset(self):
        self.componentNetwork_1 = np.array([["ampere_meter", "shortCircuit","resistor","lamp"], 
                                         ["ampere_meter", "shortCircuit","resistor","lamp"],
                                         ["ampere_meter", "switch_open","resistor","lamp"],
                                         ["ampere_meter", "volt_meter","shortCircuit","shortCircuit"]])
        self.componentNetwork_2 = np.array([["resistor","lamp"], 
                                         ["volt_meter","shortCircuit"]])
        
        
        for i in range(4):
            for j in range(4):
                self.actualChoice = [i,j]
                self.updateCircuit() 
        for i in range(1,3):
            for j in range(4,6):
                self.actualChoice = [i,j]
                self.updateCircuit() 
        self.actualChoice = [0,0]
        self.oldChoice = [0,0]
        self.updateChartChoice()
        
        for i in range(len(self.componentNetwork_1)):
            for j in range(len(self.componentNetwork_1[i])):
                self.resistancesNetwork_1[i][j] = self.dictComponentResistances[self.componentNetwork_1[i][j]]  
                
        for i in range(len(self.componentNetwork_2)):
            for j in range(len(self.componentNetwork_2[i])):
                self.resistancesNetwork_2[i][j] = self.dictComponentResistances[self.componentNetwork_2[i][j]] 
        
        self.buttonsObject.changeState("s_voltage", 230)
        
        self.callculateValues()      
        self.updateButtons()
        self.updateLabels()        
        
        self.updateGauges()
        self.updateBeams()
        self.buttonsObject.changeState("b_reset",False)
        
        
    def createComponents(self):
        layers = self.possibleComponents    
        width  = (self.width)/6
        height = (self.height)/6
        chart = Chart(layers, width, height)
                
        
        for layer in layers:
            chart.changeLayerLook(layer,"fillingColor",Color.BACKGND.value)   
            chart.changeLayerLook(layer,"lineWidth",2)   
        
            #chart.drawRectangle(layer,0,0,width,height,False)
        
        chart.changeLayerLook("nothing","lineColor",Color.BACKGND.value)
        chart.drawRectangle("nothing",0,0,width,height,True)
        
        chart.drawRectangle("resistor",width/2-width/4,height/2-height/8,width/2,height/4)
        
        chart.drawCircle("lamp",width/2,height/2,height/4)
        chart.drawLine("lamp",width/2-(height/4/np.sqrt(2)),height/2-(height/4/np.sqrt(2)),width/2+(height/4/np.sqrt(2)),height/2+(height/4/np.sqrt(2)))
        chart.drawLine("lamp",width/2+(height/4/np.sqrt(2)),height/2-(height/4/np.sqrt(2)),width/2-(height/4/np.sqrt(2)),height/2+(height/4/np.sqrt(2)))
        
        chart.changeLayerLook("lamp_broken","lineDash",[3,3])
        chart.drawCircle("lamp_broken",width/2,height/2,height/4)
        chart.drawLine("lamp_broken",width/2-(height/4/np.sqrt(2)),height/2-(height/4/np.sqrt(2)),width/2+(height/4/np.sqrt(2)),height/2+(height/4/np.sqrt(2)))
        chart.drawLine("lamp_broken",width/2+(height/4/np.sqrt(2)),height/2-(height/4/np.sqrt(2)),width/2-(height/4/np.sqrt(2)),height/2+(height/4/np.sqrt(2)))
        
        chart.drawCircle("idle",width/2-width/4,height/2,3)
        chart.drawCircle("idle",width/2+width/4,height/2,3)        
        chart.changeLayerLook("idle","lineColor",Color.BACKGND.value)
        chart.drawLine("idle",width/2-width/4+5,height/2,width/2+width/4-5,height/2)
        
        chart.drawCircle("switch_open",width/2-width/4,height/2,3)
        chart.drawCircle("switch_open",width/2+width/4,height/2,3)  
        chart.drawLine("switch_open",width/2-width/4,height/2,width/2+width/4,height/2-30)
        chart.changeLayerLook("switch_open","lineColor",Color.BACKGND.value)
        chart.drawLine("switch_open",width/2-width/4+5,height/2,width/2+width/4-5,height/2)
        
        chart.drawCircle("switch_close",width/2-width/4,height/2,3)
        chart.drawCircle("switch_close",width/2+width/4,height/2,3)  
        chart.drawLine("switch_close",width/2-width/4+5,height/2,width/2+width/4-5,height/2)
        chart.changeLayerLook("switch_close","lineDash",[3,3])
        chart.drawLine("switch_close",width/2-width/4,height/2,width/2+width/4,height/2-30)  
        
        chart.drawCircle("ampere_meter",width/2,height/2,height/4)
        chart.changeLayerLook("ampere_meter","fillingColor",Color.LINES.value)
        chart.drawText("ampere_meter",width/2,height/2+10,"30px sans-serif", "A",align = "center")
        
        chart.drawCircle("volt_meter",width/2,height/2,height/4)
        chart.changeLayerLook("volt_meter","fillingColor",Color.LINES.value)
        chart.drawText("volt_meter",width/2,height/2+10,"30px sans-serif", "V", align = "center")
        
        
        chart.update("nothing","resistor","lamp","lamp_broken","switch_open","switch_close","shortCircuit","idle","ampere_meter","volt_meter")
        
        self.componentDict = {}
        for name in layers:
            self.componentDict[name] = chart.getImage(name)
        
        #Bilder der Lampenstrahlen
        brightnessLevel = 7 #Anzahl an Helligketsstufen
        step = 7 #Anzahl an Schritten pro Helligkeitsstufe ! Muss mindestens mit der maximal unterschiedlichen Anzahl an Stromstärken übereinstimmen
        minBeams = 3 #Minimale Anzahl an Strahlen
        maxBeams = brightnessLevel*step+minBeams
        
        
        self.beamsBrightnessList = []
        for i in range(brightnessLevel):
            
            layers = list(range(minBeams+i, maxBeams+i,brightnessLevel))   
            width  = (self.width)/6
            height = (self.height)/6
            chart = Chart(layers, width, height)                

            x = width/2
            y = height/2

            for currentLayer in layers:
                
                chart.changeLayerLook(currentLayer,"fillingColor",Color.LIGHT.value)   
                chart.changeLayerLook(currentLayer,"lineColor",Color.LIGHT.value)   
                chart.changeLayerLook(currentLayer,"transparency",0.5)
                chart.changeLayerLook(currentLayer,"lineWidth",3) 

                radius = self.height/(4*6)
                chart.drawCircle(currentLayer,x,y,radius)
                
                chart.changeLayerLook(currentLayer,"lineColor",Color.BEAMS.value) 
                chart.changeLayerLook(currentLayer,"lineWidth",4)
                chart.changeLayerLook(currentLayer,"transparency",1)
                
                radiusBeams = radius +int(currentLayer)/4.5
                maxLength = height/2 - (radius +((maxBeams-1)/4.5))
                beamsLength = radiusBeams + (int(currentLayer)*np.heaviside(int(currentLayer), 1)-(int(currentLayer)-maxLength)*np.heaviside(int(currentLayer)-maxLength, 1))
                
                for i in range(int(currentLayer)):

                    phi = pi / int(currentLayer) * (2*i + 1)
                    co  = cos(phi)
                    si  = sin(phi)
                    x0  = co * radiusBeams+x
                    x1  = co * beamsLength+x
                    y0  = si * radiusBeams+y
                    y1  = si * beamsLength+y

                    chart.drawLine(currentLayer,x0,y0,x1,y1) 
                chart.update(currentLayer)
                #display(chart.getImage(currentLayer))

            beamsList = []
            for name in range(len(layers)):
                beamsList.append(chart.getImage(layers[name]))
            
            self.beamsBrightnessList.append(beamsList)
            
    def interactionHandling(self,x,y):
        column = int((x-self.border)/((self.width)/6))
        row = int((y-self.border)/(self.height/6))-2                
        if (column < 4 and row >= 0) or ((column > 3 and column < 6) and (row == 1 or row == 2)):
            self.actualChoice = [row,column]
    
    def updateChartChoice(self):
        currentLayer = "choice"
        row = self.actualChoice[0]
        column = self.actualChoice[1]
        x = (column*self.width)/6+self.border
        y = (row+2)*(self.height/6)+self.border
        
        self.chartObject.clearLayer(currentLayer)        
        self.chartObject.drawRectangle(currentLayer,x,y,self.width/6,self.height/6,False)
        self.chartObject.update(currentLayer) 
    
    def setResistanceNetworks(self):
        row = self.actualChoice[0]
        column = self.actualChoice[1]
        if self.actualChoice[1] < 4:
            self.resistancesNetwork_1[row,column] = self.buttonsStatesDict["s_resistance"]
        else:
            self.resistancesNetwork_2[row-1,column-4] = self.buttonsStatesDict["s_resistance"]
            
    def setComponentNetworks(self):
        row = self.actualChoice[0]
        column = self.actualChoice[1]
        if self.actualChoice[1] < 4:
            self.componentNetwork_1[row,column] = self.buttonsStatesDict["dd_component"]
            self.resistancesNetwork_1[row,column] = self.dictComponentResistances[self.buttonsStatesDict["dd_component"]]
        else:
            self.componentNetwork_2[row-1,column-4] = self.buttonsStatesDict["dd_component"]
            self.resistancesNetwork_2[row-1,column-4] = self.dictComponentResistances[self.buttonsStatesDict["dd_component"]]
    
    
    def callculateValues(self):  
                
            #Gesamtwiderständ der Reihen berechnen, Netzwerk 1
            for row in range(self.resistancesNetwork_1.shape[0]):
                value = 0
                for column in range(self.resistancesNetwork_1.shape[1]):
                    value = value + self.resistancesNetwork_1[row,column]
                self.resistancesSeries_1[row] = value
            #print(self.resistancesSeries_1)
            
            #Gesamtwiderständ der Reihen berechnen, Netzwerk 2
            for row in range(self.resistancesNetwork_2.shape[0]):
                value = 0
                for column in range(self.resistancesNetwork_2.shape[1]):
                    value = value + self.resistancesNetwork_2[row,column]
                self.resistancesSeries_2[row] = value
            #print(self.resistancesSeries_2)

            #Gesamtwiderstand von Netzwerk 1 berechnen
            totalResistance_1 = 0
            for value in self.resistancesSeries_1:
                totalResistance_1 = totalResistance_1 + (1/value)
            totalResistance_1 = 1/totalResistance_1            
            
            #Gesamtwiderstand von Netzwerk 2 berechnen
            totalResistance_2 = 0
            for value in self.resistancesSeries_2:
                totalResistance_2 = totalResistance_2 + (1/value)
            totalResistance_2 = 1/totalResistance_2            
            
            #Gesamtwiderstand berechnen
            self.totalResistance = totalResistance_1 + totalResistance_2 + self.lampResistance
            
            #Gesamtstrom berechnen
            self.totalCurrent = np.round(self.buttonsStatesDict["s_voltage"]/self.totalResistance,2)

            #Spannung über feste Lampe berechnen
            self.lampVoltage = self.lampResistance*self.totalCurrent
            
            #Strom und Spannung, Netzwerk 1
            totalVoltage = self.totalCurrent*totalResistance_1
            for row in range(self.voltagesNetwork_1.shape[0]):                
                for column in range(self.voltagesNetwork_1.shape[1]):
                    current = totalVoltage / self.resistancesSeries_1[row]
                    voltage = current * self.resistancesNetwork_1[row,column]
                    self.currentsNetwork_1[row,column] = current
                    self.voltagesNetwork_1[row,column] = voltage
            #print(self.currentsNetwork_1)
            #print(self.voltagesNetwork_1)
            
            #Strom und Spannung, Netzwerk 2
            totalVoltage = self.totalCurrent*totalResistance_2
            for row in range(self.voltagesNetwork_2.shape[0]):                
                for column in range(self.voltagesNetwork_2.shape[1]):
                    current = totalVoltage / self.resistancesSeries_2[row]
                    voltage = current * self.resistancesNetwork_2[row,column]
                    self.currentsNetwork_2[row,column] = current
                    self.voltagesNetwork_2[row,column] = voltage
            #print(self.currentsNetwork_2)
            #print(self.voltagesNetwork_2)
            
            
            np.round(self.totalCurrent,2)
            np.round(self.lampVoltage,2)
            np.round(self.currentsNetwork_1,2)
            np.round(self.voltagesNetwork_1,2)
            
            np.round(self.currentsNetwork_2,2)
            np.round(self.voltagesNetwork_2,2)
            
            
    def updateCircuit(self):
        
        row = self.actualChoice[0]
        column = self.actualChoice[1]
        
        #Zugehörige Ebene         
        currentLayer = str(row)+"-"+str(column)

        #Koordinaten der Ebene
        x = self.width/6*(column)
        y = self.height/6*(row+2)   
        if column<4:
            component = self.componentNetwork_1[row,column]
        else:
            component = self.componentNetwork_2[row-1,column-4]           
 
        self.chartObject.clearLayer(currentLayer)
        self.chartObject.drawImage(currentLayer,self.componentDict[component],x+self.border,y+self.border)        
        self.chartObject.update(currentLayer) 
    
    def updateButtons(self):
        
        row = self.actualChoice[0]
        column = self.actualChoice[1]
        
        if column < 4:
            component = self.componentNetwork_1[row,column]
            resistance = self.resistancesNetwork_1[row,column]
        else:
            component = self.componentNetwork_2[row-1,column-4]
            resistance = self.resistancesNetwork_2[row-1,column-4]

            
        if component != "resistor":
            self.buttonsObject.disabled("s_resistance", True)
        else:
            self.buttonsObject.disabled("s_resistance", False)
        
        self.buttonsObject.changeState("s_resistance", resistance)
        self.buttonsObject.changeState("dd_component", component)

        
    def updateLabels(self):
        
        row = self.actualChoice[0]
        column = self.actualChoice[1]        
        if  column < 4:
            voltage = self.voltagesNetwork_1[row,column]
            current = self.currentsNetwork_1[row,column]
        else:
            voltage = self.voltagesNetwork_2[row-1,column-4]
            current = self.currentsNetwork_2[row-1,column-4]
        
        self.currentValueLabel.value = str(round(current,2))
        self.voltageValueLabel.value = str(round(voltage,2))
            
    def updateBeams(self):
        
        #Liste mit allen Spannungswerten der Lampen erstellen und Apseichern zugehöriger Koordinaten
        currentsLamps = []
        koordinateLamps = []
        
        list_row,list_column = np.where(self.componentNetwork_1 == "lamp")
        for i in range(len(list_row)):
            currentsLamps.append(self.currentsNetwork_1[list_row[i],list_column[i]])
            x = self.width/6*(list_column[i])+self.border 
            y = self.height/6*(list_row[i]+2)+self.border 
            koordinateLamps.append([x,y])
            
        list_row,list_column = np.where(self.componentNetwork_2 == "lamp")
        for i in range(len(list_row)):
            currentsLamps.append(self.currentsNetwork_2[list_row[i],list_column[i]])
            x = self.width/6*(list_column[i]+4)+self.border 
            y = self.height/6*(list_row[i]+3)+self.border 
            koordinateLamps.append([x,y])
            
        
        currentsLamps.append(self.totalCurrent )
        koordinateLamps.append([self.width/6*3+self.border,self.border])
        
        tuples = zip(*sorted(zip(currentsLamps, koordinateLamps)))
        currentsLamps, koordinateLamps = [ list(tuple) for tuple in  tuples]        
        
        #print(voltageLamps, koordinateLamps)
        currentLayer = "beams"
        self.chartObject.clearLayer(currentLayer)
        
        
        j = 0
        for i in range(len(currentsLamps)):
            current = np.round(currentsLamps[i],2)
            
            #Auswahl der Helligkeitsstufenliste
            if current > 0:                
                number = int((np.round((np.heaviside(np.log10(current),1)*np.log10(current)*100),0)))
                if number < len(self.beamsBrightnessList):
                    beamsList = (self.beamsBrightnessList[number])
                else:
                    beamsList = (self.beamsBrightnessList[-1])
            
                #Auswahl des Strahlenbildes
                if (i > 0 and current != np.round(currentsLamps[i-1],2)):
                    j+=1   
                    
                img = beamsList[j]
                

                #Suchen der Koordinaten 
                x = koordinateLamps[i][0]
                y = koordinateLamps[i][1]
                #Zeichnen des Bildes
                self.chartObject.drawImage(currentLayer,img,x,y)
        self.chartObject.update(currentLayer)    
            
        
        
    def updateGauges(self):
        
        currentLayer = "gauges"
        self.chartObject.clearLayer(currentLayer)
        
        x_correction = self.width/6*0.5+self.border
        y_correction = self.height/6*0.5+self.border 
        
        
        #Strom und Spannung Erstes Netzwerk
        
        list_row,list_column = np.where(self.componentNetwork_1 == "ampere_meter")
        
        for i in range(len(list_row)):
            x = self.width/6*(list_column[i])
            y = self.height/6*(list_row[i]+2)
            self.chartObject.drawText(currentLayer,x+x_correction,y+y_correction/2,"20px sans-serif", str(round(self.currentsNetwork_1[list_row[i],list_column[i]],2))+"A",align = "center")
        

        list_row,list_column = np.where(self.componentNetwork_1 == "volt_meter")
        for i in range(len(list_row)):
            x = self.width/6*(list_column[i])
            y = self.height/6*(list_row[i]+2)
            self.chartObject.drawText(currentLayer,x+x_correction,y+y_correction/2,"20px sans-serif", str(round(self.voltagesNetwork_1[list_row[i],list_column[i]],2))+"V",align = "center")
        
        
        #Strom und Spannung Erstes Netzwerk
        
        list_row,list_column = np.where(self.componentNetwork_2 == "ampere_meter")
        
        for i in range(len(list_row)):
            x = self.width/6*(list_column[i]+4)
            y = self.height/6*(list_row[i]+3)
            self.chartObject.drawText(currentLayer,x+x_correction,y+y_correction/2,"20px sans-serif", str(round(self.currentsNetwork_2[list_row[i],list_column[i]],2))+"A",align = "center")
        

        list_row,list_column = np.where(self.componentNetwork_2 == "volt_meter")
        for i in range(len(list_row)):
            x = self.width/6*(list_column[i]+4)
            y = self.height/6*(list_row[i]+3)
            self.chartObject.drawText(currentLayer,x+x_correction,y+y_correction/2,"20px sans-serif", str(round(self.voltagesNetwork_2[list_row[i],list_column[i]],2))+"V",align = "center")
       
       
        
        #Strom und Spannung bei oberer Lampe anzeigen
        self.chartObject.drawText(currentLayer,self.width/12*5.5+x_correction,y_correction/2,"20px sans-serif", str(round(self.totalCurrent,2))+"A",align = "center")
        self.chartObject.drawText(currentLayer,self.width/12*6.5+x_correction,y_correction/2,"20px sans-serif", str(round(self.lampVoltage,2))+"V",align = "center")
        
        self.chartObject.update(currentLayer)
        

In [3]:
if __name__ == '__main__':
    a = Widerstandsnetzwerk(None)    
    a.visualize()
    a.start()
